In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images
print(train_features.shape)
print(valid_features.shape)
print(test_features.shape)
print(type(train_features))

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1,k,k,1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k = 2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k = 2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28,28,1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

#Model
logits = conv_net(x, weights, biases, keep_prob)

#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            
        print('Epoch {:>2},Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1, loss, valid_acc))
            
     # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))        

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz
(55000, 28, 28, 1)
(5000, 28, 28, 1)
(10000, 28, 28, 1)
<class 'numpy.ndarray'>
Epoch  1,Loss:  1900.8705 Validation Accuracy: 0.746094
Epoch  2,Loss:   956.2587 Validation Accuracy: 0.820312
Epoch  3,Loss:   422.5016 Validation Accuracy: 0.847656
Epoch  4,Loss:   534.4116 Validation Accuracy: 0.847656
Epoch  5,Loss:   346.2297 Validation Accuracy: 0.867188
Epoch  6,Loss:   490.7515 Validation Accuracy: 0.871094
Epoch  7,Loss:   191.4990 Validation Accuracy: 0.875000
Epoch  8,Loss:   345.5573 Validation Accuracy: 0.875000
Epoch  9,Loss:   258.5102 Validation Accuracy: 0.875000
Epoch 10,Loss:   246.9955 Validation Accuracy: 0.875000
Testing Accuracy: 0.890625


In [35]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape = shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict = {xs: v_xs, keep_prob:1.0})
    correct_prection = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prection, tf.float32))
    result = sess.run(accuracy, feed_dict = {xs: v_xs, ys: v_ys})
    return result

xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28,28,1])

w_conv1 = weight_variable([5,5,1,32]) #patch:5x5  input size:1  output size: 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(tf.add(conv2d(x_image,w_conv1),b_conv1)) #output size: 28x28x32
h_pool1 = max_pool(h_conv1) #output size: 14x14x32

w_conv2 = weight_variable([5,5,32,64]) #patch 5x5, input size:32 output size: 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(tf.add(conv2d(h_pool1, w_conv2),b_conv2)) #output size: 14x14x32
h_pool2 = max_pool(h_conv2) #output size: 7x7x64

## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                             reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1001):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        if i%50 == 0:
            print("i:",i, "accuracy:", compute_accuracy(mnist.test.images[:1000], mnist.test.labels[:1000]))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
i: 0 accuracy: 0.094
i: 50 accuracy: 0.758
i: 100 accuracy: 0.864
i: 150 accuracy: 0.892
i: 200 accuracy: 0.913
i: 250 accuracy: 0.926
i: 300 accuracy: 0.929
i: 350 accuracy: 0.936
i: 400 accuracy: 0.942
i: 450 accuracy: 0.941
i: 500 accuracy: 0.948
i: 550 accuracy: 0.946
i: 600 accuracy: 0.951
i: 650 accuracy: 0.956
i: 700 accuracy: 0.957
i: 750 accuracy: 0.96
i: 800 accuracy: 0.961
i: 850 accuracy: 0.963
i: 900 accuracy: 0.967
i: 950 accuracy: 0.963
i: 1000 accuracy: 0.969
